In [30]:
import torch

In [31]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel


query = "что должны обеспечивать работники железнодорожного транспорта?"

docs = [
    "Работники железнодорожного транспорта в соответствии со своими должностными обязанностями должны обеспечивать \
    выполнение Правил и приложений к ним, безопасность движения и эксплуатации железнодорожного транспорта.", 
    "Ответственными за содержание и (или) исправное техническое состояние железнодорожных путей, сооружений и устройств железнодорожного транспорта \
    с обеспечением периодичности выполнения ремонтов, установленных нормативной технической документацией, являются работники железнодорожного транспорта, \
    непосредственно их обслуживающие.",
    "Работники железнодорожного транспорта обязаны соблюдать правила и нормы по охране труда, промышленной, экологической, \
    пожарной безопасности, санитарно-эпидемиологические правила и нормативы в соответствии со своими должностными обязанностями и должностными инструкциями.",
    "Доступ на локомотивы, в кабины управления мотор-вагонного подвижного состава <18>, к специальному самоходному подвижному составу <19>, \
    к сигналам, железнодорожным стрелкам <20> (далее - стрелка), аппаратам, механизмам и другим устройствам, связанным с обеспечением безопасности движения \
    и эксплуатации железнодорожного транспорта, в помещения, из которых производится управление сигналами и указанными устройствами, имеют работники железнодорожного \
    транспорта, в случае, если нахождение работников железнодорожного транспорта на указанных объектах предусмотрено их должностными обязанностями. Запрещается доступ \
    посторонних лиц на указанные в настоящем пункте объекты.",
    "В соответствии с пунктом 3 статьи 25 Федерального закона 'О железнодорожном транспорте в Российской Федерации' <25> лица, принимаемые на работу, непосредственно \
    связанную с движением поездов и маневровой работой, и работники, выполняющие такую работу и (или) подвергающиеся воздействию вредных и опасных производственных факторов, \
    проходят за счет средств работодателей обязательные предварительные (при поступлении на работу) и периодические (в течение трудовой деятельности) медицинские осмотры, \
    включающие в себя химико-токсикологические исследования наличия в организме человека наркотических средств, психотропных веществ и их метаболитов в соответствии с Порядком \
    проведения обязательных предварительных (при поступлении на работу) и периодических (в течение трудовой деятельности) медицинских осмотров на железнодорожном транспорте, \
    утвержденным приказом Министерства транспорта Российской Федерации от 19 октября 2020 г. N 428 <26>.",
    "Работники железнодорожного транспорта, производственная деятельность которых связана с движением поездов \
    и маневровой работой на железнодорожных путях общего пользования, должны проходить аттестацию, предусматривающую проверку \
    знаний Правил, инструкций по организации движения поездов и маневровой работы, по сигнализации на железнодорожном транспорте, \
    и иных нормативных правовых актов федерального органа исполнительной власти в области железнодорожного транспорта <29>",
    ]

docs = [doc.lower() for doc in docs]

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

#Tokenize sentences
query_tok = tokenizer(query, padding=True, truncation=True, max_length=24, return_tensors='pt')
doc_tok = tokenizer(docs, padding=True, truncation=True, max_length=24, return_tensors='pt')

with torch.no_grad():
    query_emb = model(**query_tok)
    doc_emb = model(**doc_tok)

query_emb = mean_pooling(query_emb, query_tok['attention_mask'])
doc_emb = mean_pooling(doc_emb, doc_tok['attention_mask'])

# --------------------------------------------------------------------

#Compute dot score between query and all document embeddings
scores = util.cos_sim(query_emb, doc_emb)[0]

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score.item(), doc)

0.6893495321273804 работники железнодорожного транспорта в соответствии со своими должностными обязанностями должны обеспечивать     выполнение правил и приложений к ним, безопасность движения и эксплуатации железнодорожного транспорта.
0.6452568173408508 работники железнодорожного транспорта обязаны соблюдать правила и нормы по охране труда, промышленной, экологической,     пожарной безопасности, санитарно-эпидемиологические правила и нормативы в соответствии со своими должностными обязанностями и должностными инструкциями.
0.5124560594558716 ответственными за содержание и (или) исправное техническое состояние железнодорожных путей, сооружений и устройств железнодорожного транспорта     с обеспечением периодичности выполнения ремонтов, установленных нормативной технической документацией, являются работники железнодорожного транспорта,     непосредственно их обслуживающие.
0.5043462514877319 работники железнодорожного транспорта, производственная деятельность которых связана с движение

In [32]:
# tokenizer.save_pretrained('tokenizer')

In [33]:
# model.save_pretrained('model')

In [34]:
def cos_similarity(first_embeddings: torch.Tensor, second_embeddings: torch.Tensor) -> torch.Tensor:
    return util.cos_sim(first_embeddings, second_embeddings)

In [35]:
def mean_pooling(model_output: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [36]:
from typing import List

def get_embedding(text: str | List[str], tokenizer, model) -> torch.Tensor:
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=16, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input)

    return mean_pooling(model_output, encoded_input['attention_mask'])

In [37]:
def lower_case(text: str) -> str:
    return text.lower()

In [38]:
from typing import Dict


def find_sentences(query: str, id2text: Dict[int, str], id2vec: torch.Tensor, tokenizer, model) -> str:
    query_emb = get_embedding(lower_case(query), tokenizer, model)
    return id2text[torch.argmax(cos_similarity(query_emb, id2vec)).item()]

In [39]:
def find_top_k_sentences(query: str, id2text: Dict[int, str], id2vec: torch.Tensor, tokenizer, model, top_k: int = 3) -> List[str]:
    query_emb = get_embedding(lower_case(query), tokenizer, model)
    idxs = torch.topk(cos_similarity(query_emb, id2vec), top_k).indices[0]
    return [id2text[id.item()] for id in idxs]

## Чтение доков

In [40]:
from os import listdir

my_path = 'dataset'

path_tree = {
    my_path : dict()
}


for file_name in listdir(my_path):
    if '.txt' in file_name:
        path_tree[my_path][file_name] = 0
    else:
        path_tree[my_path][file_name] = dict()
        for second_file_name in listdir(f'{my_path}/{file_name}'):
            path_tree[my_path][file_name][second_file_name] = 0



In [41]:
path_tree

{'dataset': {'1': {'1.1.txt': 0},
  '10': {'10.1.txt': 0, '10.2.txt': 0, '10.3.txt': 0},
  '11': {'11.1.txt': 0,
   '11.2.txt': 0,
   '11.3.txt': 0,
   '11.4.txt': 0,
   '11.5.txt': 0,
   '11.6.txt': 0,
   '11.7.txt': 0},
  '12': {'12.1.txt': 0, '12.2.txt': 0, '12.3.txt': 0, '12.4.txt': 0},
  '13': {'13.1.txt': 0, '13.2.txt': 0, '13.3.txt': 0, '13.4.txt': 0},
  '14': {'14.1.txt': 0,
   '14.2.txt': 0,
   '14.3.txt': 0,
   '14.4.txt': 0,
   '14.5.txt': 0,
   '14.6.txt': 0,
   '14.7.txt': 0,
   '14.8.txt': 0,
   '14.9.txt': 0},
  '15': {'15.1.txt': 0,
   '15.2.txt': 0,
   '15.3.txt': 0,
   '15.4.txt': 0,
   '15.5.txt': 0},
  '16.txt': 0,
  '17.txt': 0,
  '18.txt': 0,
  '19.txt': 0,
  '2': {'2.1.txt': 0,
   '2.10.txt': 0,
   '2.11.txt': 0,
   '2.2.txt': 0,
   '2.3.txt': 0,
   '2.4.txt': 0,
   '2.5.txt': 0,
   '2.6.txt': 0,
   '2.7.txt': 0,
   '2.8.txt': 0,
   '2.9.txt': 0},
  '20.txt': 0,
  '21.txt': 0,
  '22.txt': 0,
  '23.txt': 0,
  '3': {'3.1.txt': 0, '3.2.txt': 0, '3.3.txt': 0, '3.4.tx

In [42]:
import re

def parser(path: str) -> List[str]:
    with open(path, 'r', encoding = 'cp1251') as file:
        s = file.read()
        s = re.sub(" <\d+>", "", s)
        s = re.sub("<\d+> ", "", s)
        s = re.split("\n\d+\. ", s)
        s = [re.sub("\n", " ", line) for line in s]
        s = [re.sub("  ", "\n", line) for line in s]
        s[0] = s[0][3:]
        return s

## Создание текстового корпуса

In [43]:
text_corpus = []

for file_name, value in path_tree[my_path].items():
    next_path = f'{my_path}/{file_name}'
    if value == 0:
        text_corpus.extend(parser(next_path))
    else:
        for second_file_name in listdir(next_path):
            text_corpus.extend(parser(f'{next_path}/{second_file_name}'))


In [44]:
from joblib import dump


# dump(text_corpus, 'text_corpus.pkl')

## Сделать эмбеддинги

In [51]:
from joblib import load

In [55]:
id2text = load('id2text.pkl')
id2vec = load('id2vec.pkl')

In [ ]:
# id2text = {id: text for id, text in enumerate(text_corpus)}
# id2vec = torch.vstack([get_embedding(lower_case(id2text[id]), tokenizer, model) for id in id2text.keys()])

In [ ]:
# dump(id2text, 'id2text.pkl')

In [ ]:
# dump(id2text, 'id2vec.pkl')

In [56]:
id2vec.size()

torch.Size([917, 1024])

## Тестирование

In [69]:
id = 22
id2text[id]

'Не допускается ставить в поезда:\n1) вагоны с неисправностями, угрожающими безопасности движения, указанными в Правилах, а также вагоны, состояние которых не обеспечивает сохранности перевозимых грузов;\n2) вагоны, загруженные сверх их грузоподъемности;\n3) вагоны, загруженные с нарушением технических условий размещения и крепления грузов;\n4) вагоны, имеющие просевшие рессоры, вызывающие перекос кузова или удары рамы и кузова вагона о ходовые части, а также вагоны с неисправностью кровли, создающей опасность отрыва ее листов;\n5) вагоны, не имеющие трафарета о производстве установленных видов ремонта, за исключением вагонов, следующих по перевозочным документам как груз на своих осях;\n6) вагоны - платформы, транспортеры железнодорожные и полувагоны с негабаритными грузами, если о возможности следования таких вагонов не будет дано указаний порядком, установленным владельцем инфраструктуры (владельцем железнодорожных путей необщего пользования);\n7) вагоны - платформы с незакрытыми бо

In [100]:
query = 'Работники железнодорожного транспорта, не прошедшие аттестацию, не допускаются к выполнению определенных работ.'
print(find_sentences(query, id2text, id2vec, tokenizer, model))

Работники железнодорожного транспорта в соответствии со своими должностными обязанностями должны обеспечивать выполнение Правил и приложений к ним, безопасность движения и эксплуатации железнодорожного транспорта.
Соблюдение требований Правил работниками железнодорожного транспорта обеспечивается организациями железнодорожного транспорта и индивидуальными предпринимателями, выполняющими функции работодателя по отношению к указанным работникам. 


In [97]:
article_text = ' '.join(find_top_k_sentences(query, id2text, id2vec, tokenizer, model, 5))

In [98]:
print(article_text)

Работник, переводящий стрелки, после каждого перевода стрелки должен убедиться в правильности положения остряков по индикации на пульте местного управления или по положению остряков стрелки.
Нецентрализованные стрелки, кроме расположенных на сортировочных железнодорожных путях, железнодорожных путях, где маневровая работа постоянно осуществляется толчками, и стрелок, оборудованных шарнирно-коленчатыми замыкателями, должны при маневрах запираться на закладки.  Перед переводом централизованной стрелки обслуживающий ее работник должен убедиться (лично или по докладу работника, уполномоченного владельцем инфраструктуры (владельцем железнодорожных путей необщего пользования) в том, что она не занята железнодорожным подвижным составом (фактическое ее свободное состояние) и железнодорожный подвижной состав не находится за пределами смежных железнодорожных путей. При электрической централизации отсутствие железнодорожного подвижного состава на стрелочном переводе устанавливается по индикации н

## Суммаризация

In [84]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer_sum = AutoTokenizer.from_pretrained(model_name)
model_sum = T5ForConditionalGeneration.from_pretrained(model_name)

In [105]:
def summarization(text: str, tokenizer, model):
    input_ids = tokenizer(
        [text],
        max_length=300,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    return tokenizer.decode(output_ids, skip_special_tokens=True)

In [106]:
article_text = query + article_text 
summarization(article_text, tokenizer_sum, model_sum)

'Работники железнодорожного транспорта, не прошедшие аттестацию, не допускаются к выполнению определенных работ. Перед переводом централизованной стрелки обслуживающий ее работник должен убедиться в правильности положения остряков стрелки.'